In [1]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


Using TensorFlow backend.


In [2]:
# %env CUDA_DEVICE_ORDER=PCI_BUS_ID
# %env CUDA_VISIBLE_DEVICES=0

In [3]:
ft = fastText.load_model("../../fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [4]:
import csv, re, numpy as np
user_clusters = {}
with open('user_clusters_k50.csv', 'r', encoding='UTF-8') as f:
    userreader = csv.reader(f)
    next(userreader)
    for row in userreader:
        # print(row)
        user_clusters[row[1]] = int(row[2])

In [46]:
def get_user_features(tweet):
    usermentions = [re.sub('@', '', u) for u in re.findall('@[\w_]+', tweet)]
    uvec = np.zeros(51)
    for u in usermentions:
        if u in user_clusters:
            # print(user_clusters[u])
            uvec[user_clusters[u]] += 1
        #else:
        #    print("who is" + u)
    return(uvec)
print(get_user_features("Ey @spiegel, @csu, @realdonaldtrump, @anna, @cdunrw du nervst"))

[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]


In [6]:
for i, l in enumerate(model.layers):
    print(str(i) + ' ' + l.name)

NameError: name 'model' is not defined

In [7]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [8]:
word_vectors_ft = {}
def process_features_user(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    features_user = get_user_features(textline)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft, features_user

In [9]:
def sequential_generator_user(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename, encoding='utf-8')
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_features_user = np.zeros((batch_size, 51))
#         print(type(labels2Idx))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i], batch_features_user[i] = process_features_user(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft, batch_features_user], batch_labels)

In [10]:
def train_dev_sentences(filetrain,filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain, encoding='utf-8') if len(line.strip().split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev, encoding='utf-8') if len(line.strip().split('\t'))==check]

    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
#     train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
#     dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)


In [12]:
n_labels = 2

In [13]:
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(filetrain='../../Resources/OffLang/sample_train.txt',
                   filedev='../../Resources/OffLang/sample_dev.txt', check=3)

In [14]:
print(dev_sentences[0])
print(dev_labels[:20])

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]


In [31]:
def compile_model(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
#     new_model.summary()
    return new_model

In [56]:
def test_model_tl_user(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
    f = open(filename_to_log, 'w', encoding='utf-8')
    
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(2)
        model.load_weights('classification_model_tl_twitterclasses7m_1000_suf_td.h5')

        for l in model.layers:
            l.trainable = False
        model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
        
        input1 = model.input
        input2 = Input(shape=[51], dtype='float32', name='user_input')

        model.layers.pop()
        model.summary()
        output = concatenate([model.layers[-1].output, input2])
        output = Dense(100)(output)
        output = LeakyReLU()(output)
        output = Dense(2, activation = 'softmax')(output)
        final_model = Model(inputs = [input1, input2], outputs = [output])

        final_model.compile(loss='categorical_crossentropy', optimizer=Nadam(lr=0.0001), metrics=['accuracy'])
        
        for l in final_model.layers:
            print(l.trainable)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
#         checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
#                                      save_weights_only = True)


        max_f1=0
        max_p=0
        max_r=0
        max_a=0

        
        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            final_model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx, tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1
            )

            testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
            user_features = np.zeros((len(dev_sentences), 51))
            for i in range(len(dev_sentences)):
                tmp = process_features_user(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
                testset_features[i] = tmp[0]
                user_features[i] = tmp[1]
            results = final_model.predict([testset_features, user_features])


#             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
            predLabels = results.argmax(axis=-1)
            devLabels = devLabels
            f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
            r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            a = accuracy_score(devLabels, predLabels)
            if max_f1 < f1:
                print("model saved. F1 is %f" %(f1))
                final_model.save(filename_to_save_weigths)
                max_f1 = f1
                max_p = p
                max_r = r
                max_a = a
            text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
            print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    f.write(final_text)
    f.close()



In [ ]:
from keras.models import load_model
from keras.optimizers import Nadam
# EXP USER
batch_size=32
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=3
test_model_tl_user(generator=sequential_generator_user, 
           train_sentences=train_sentences, 
           devLabels=dev_labels, 
           number_of_tests= 10,
           number_of_epochs=50, 
           filename_to_log='results_user_feature.txt',
           labels2Idx = labels2Idx,
           filename_to_save_weigths='classification_model_user_feature.h5',
           batch_size=batch_size,
           train_file='../../Resources/OffLang/sample_train.txt', 
           f1_measure='binary', 
           pos_label=1, 
           load_model_weights=True,
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/10
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_18 (Bidirectional (None, 75, 200)      320800      input_18[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_103 (LeakyReLU)     (None, 75, 200)      0           bidirectional_18[0][0]           
__________________________________________________________________________________________________
conv1d_52 (Conv1D)              (None, 73, 200)      120200      leaky_re_lu_103[0][0]            
__________________________________________________________________________________________________


Epoch 1/1
132/132 [==============================] - 18s 134ms/step - loss: 0.5351 - acc: 0.7576
model saved. F1 is 0.706740
Test-Data: Prec: 0.688, Rec: 0.727, F1: 0.707, Acc: 0.801
Epoch: 2
Epoch 1/1
132/132 [==============================] - 14s 104ms/step - loss: 0.2183 - acc: 0.9096
model saved. F1 is 0.735883
Test-Data: Prec: 0.716, Rec: 0.757, F1: 0.736, Acc: 0.821
Epoch: 3
Epoch 1/1
132/132 [==============================] - 13s 100ms/step - loss: 0.1780 - acc: 0.9304
model saved. F1 is 0.746429
Test-Data: Prec: 0.713, Rec: 0.783, F1: 0.746, Acc: 0.824
Epoch: 4
Epoch 1/1
132/132 [==============================] - 13s 100ms/step - loss: 0.1470 - acc: 0.9392
model saved. F1 is 0.748663
Test-Data: Prec: 0.714, Rec: 0.787, F1: 0.749, Acc: 0.825
Epoch: 5
Epoch 1/1
132/132 [==============================] - 13s 101ms/step - loss: 0.1327 - acc: 0.9453
model saved. F1 is 0.757143
Test-Data: Prec: 0.724, Rec: 0.794, F1: 0.757, Acc: 0.832
Epoch: 6
Epoch 1/1
132/132 [=====================

132/132 [==============================] - 13s 102ms/step - loss: 0.0978 - acc: 0.9619
Test-Data: Prec: 0.697, Rec: 0.801, F1: 0.746, Acc: 0.819
Epoch: 21
Epoch 1/1
132/132 [==============================] - 13s 101ms/step - loss: 0.0962 - acc: 0.9598
Test-Data: Prec: 0.710, Rec: 0.790, F1: 0.748, Acc: 0.824
Epoch: 22
Epoch 1/1
132/132 [==============================] - 13s 101ms/step - loss: 0.0908 - acc: 0.9645
Test-Data: Prec: 0.709, Rec: 0.787, F1: 0.746, Acc: 0.823
Epoch: 23
Epoch 1/1
132/132 [==============================] - 13s 102ms/step - loss: 0.0909 - acc: 0.9640
Test-Data: Prec: 0.704, Rec: 0.794, F1: 0.746, Acc: 0.822
Epoch: 24
Epoch 1/1
132/132 [==============================] - 14s 102ms/step - loss: 0.0939 - acc: 0.9602
Test-Data: Prec: 0.713, Rec: 0.783, F1: 0.746, Acc: 0.824
Epoch: 25
Epoch 1/1
132/132 [==============================] - 14s 102ms/step - loss: 0.0800 - acc: 0.9695
Test-Data: Prec: 0.703, Rec: 0.798, F1: 0.747, Acc: 0.822
Epoch: 26
Epoch 1/1
132/132 [=

Epoch 1/1
132/132 [==============================] - 19s 144ms/step - loss: 0.3425 - acc: 0.8435
model saved. F1 is 0.727915
Test-Data: Prec: 0.689, Rec: 0.772, F1: 0.728, Acc: 0.809
Epoch: 2
Epoch 1/1
132/132 [==============================] - 14s 104ms/step - loss: 0.1891 - acc: 0.9205
model saved. F1 is 0.736111
Test-Data: Prec: 0.686, Rec: 0.794, F1: 0.736, Acc: 0.812
Epoch: 3
Epoch 1/1
132/132 [==============================] - 13s 100ms/step - loss: 0.1455 - acc: 0.9387
model saved. F1 is 0.737030
Test-Data: Prec: 0.705, Rec: 0.772, F1: 0.737, Acc: 0.818
Epoch: 4
Epoch 1/1
132/132 [==============================] - 13s 98ms/step - loss: 0.1219 - acc: 0.9510
Test-Data: Prec: 0.703, Rec: 0.772, F1: 0.736, Acc: 0.817
Epoch: 5
Epoch 1/1
132/132 [==============================] - 13s 98ms/step - loss: 0.1243 - acc: 0.9503
model saved. F1 is 0.740741
Test-Data: Prec: 0.700, Rec: 0.787, F1: 0.741, Acc: 0.818
Epoch: 6
Epoch 1/1
132/132 [==============================] - 13s 98ms/step - l

132/132 [==============================] - 13s 97ms/step - loss: 0.0969 - acc: 0.9619
Test-Data: Prec: 0.689, Rec: 0.813, F1: 0.746, Acc: 0.817
Epoch: 21
Epoch 1/1
132/132 [==============================] - 13s 98ms/step - loss: 0.0913 - acc: 0.9640
Test-Data: Prec: 0.696, Rec: 0.805, F1: 0.747, Acc: 0.819
Epoch: 22
Epoch 1/1
132/132 [==============================] - 13s 99ms/step - loss: 0.0939 - acc: 0.9643
Test-Data: Prec: 0.720, Rec: 0.779, F1: 0.748, Acc: 0.827
Epoch: 23
Epoch 1/1
132/132 [==============================] - 13s 99ms/step - loss: 0.0819 - acc: 0.9666
Test-Data: Prec: 0.704, Rec: 0.801, F1: 0.750, Acc: 0.823
Epoch: 24
Epoch 1/1
132/132 [==============================] - 13s 100ms/step - loss: 0.0906 - acc: 0.9616
model saved. F1 is 0.750442
Test-Data: Prec: 0.711, Rec: 0.794, F1: 0.750, Acc: 0.825
Epoch: 25
Epoch 1/1
132/132 [==============================] - 13s 99ms/step - loss: 0.0935 - acc: 0.9647
model saved. F1 is 0.752228
Test-Data: Prec: 0.718, Rec: 0.790, F

Epoch 1/1
132/132 [==============================] - 18s 138ms/step - loss: 0.4715 - acc: 0.7884
model saved. F1 is 0.729204
Test-Data: Prec: 0.691, Rec: 0.772, F1: 0.729, Acc: 0.811
Epoch: 2
Epoch 1/1
132/132 [==============================] - 13s 100ms/step - loss: 0.1959 - acc: 0.9176
Test-Data: Prec: 0.684, Rec: 0.779, F1: 0.729, Acc: 0.808
Epoch: 3
Epoch 1/1
  3/132 [..............................] - ETA: 13s - loss: 0.1125 - acc: 0.9479